In [2]:
import pandas as pd
import os
from pyproj import Transformer
import pandas as pd
import folium
from folium.plugins import MarkerCluster, HeatMap
import numpy as np
import seaborn as sns

# 1. Limpiar datos erróneos en dataset de Autobuses EMT

## Convertir Separador CSV - Autobuses EMT

Este notebook convierte el archivo `stopsemt.csv` de separador coma (`,`) a punto y coma (`;`) para evitar problemas con campos que contienen comas dentro de comillas.

**Input**: `data/AUTOBUSES/stopsemt.csv` (separador: `,`)

**Output**: `data/AUTOBUSES/stopsemt_clean.csv` (separador: `;`)

In [3]:
df = pd.read_csv("data/AUTOBUSES/stopsemt.csv", sep=",", quotechar='"')
df.to_csv("data/AUTOBUSES/stopsemt_clean.csv", sep=";", index=False, quotechar='"')

## Conversión de Coordenadas UTM a Lat/Long
- **Sistema origen**: UTM Zone 30N (EPSG:25830) - coordenadas proyectadas en metros
- **Sistema destino**: WGS84 (EPSG:4326) - coordenadas geográficas en grados decimales
- **Transformación**: Usa `pyproj.Transformer` para conversión precisa
- **Campos afectados**: `posX` (Este → Longitud), `posY` (Norte → Latitud)

In [4]:
autobuses = pd.read_csv('data/AUTOBUSES/stopsemt_clean.csv', sep=';')

transformer = Transformer.from_crs("EPSG:25830", "EPSG:4326", always_xy=True)

lon, lat = transformer.transform(
    autobuses['posX'].values, 
    autobuses['posY'].values
)

autobuses['posX'] = lon
autobuses['posY'] = lat

autobuses.to_csv('data/AUTOBUSES/stopsemt_clean_coordinates_converted.csv', sep=';', index=False)

# Cargar los datos limpios (salida de hop)

In [9]:
df_metro = pd.read_csv('resultados/metro_procesado.csv', sep=';')
df_autobuses = pd.read_csv('resultados/autobuses_procesado.csv', sep=';')
df_bicimad = pd.read_csv('resultados/bicimad_procesado.csv', sep=';')
df_parkings = pd.read_csv('resultados/parkings_procesado.csv', sep=';')

df_all = pd.read_csv('resultados/transporte_madrid_consolidado.csv', sep=';')

In [6]:
df_metro.head()

,stop_id,stop_name,stop_lat,stop_lon,transport_mode
0,par_4_1,PLAZA DE CASTILLA,40.46690,-3.68917,metro
1,acc_4_1_1,Plaza de Castilla,40.46682,-3.68918,metro
2,acc_4_1_1040,Ascensor,40.46555,-3.68877,metro
3,acc_4_1_1043,Intercambiador Superficie,40.46728,-3.68915,metro
4,acc_4_1_1044,Ascensor,40.46702,-3.68918,metro


In [7]:
df_autobuses.head()

,stop_id,stop_name,bus_line,stop_lon,stop_lat,transport_mode
0,4514,Cristo Rey ...,1,-3.716655,40.440258,bus
1,4022,Junta Municipal Moncloa ...,1,-3.717145,40.437624,bus
2,3687,Moncloa ...,1,-3.716809,40.435924,bus
3,737,Altamirano ...,1,-3.716282,40.433990,bus
4,735,Argüelles ...,1,-3.714798,40.431936,bus


In [8]:
df_bicimad.head()

,station_id,station_name,stop_lon,stop_lat,state,capacity,transport_mode
0,2,Metro Callao,-3.705690,40.420400,IN_SERVICE,27,bicimad
1,3,Plaza Conde Suchil,-3.707254,40.430322,IN_SERVICE,19,bicimad
2,4,Malasaña,-3.702500,40.428626,IN_SERVICE,27,bicimad
3,5,Fuencarral,-3.702135,40.428521,IN_SERVICE,27,bicimad
4,6,Colegio de Arquitectos,-3.699023,40.423178,IN_SERVICE,19,bicimad


In [9]:
df_parkings.head()

,parking_id,parking_name,stop_lat,stop_lon,standard_spaces,pmr_spaces,PMR_ratio,transport_mode
0,2,Colón,40.424709,-3.689939,1047,21,0.020057,parking
1,3,Corazón de María II,40.438525,-3.645525,327,0,0.000000,parking
2,4,Encuentro,40.405465,-3.651354,104,0,0.000000,parking
3,5,Nuestra Señora del Recuerdo,40.472181,-3.679160,902,12,0.013304,parking
4,6,Corona Boreal,40.456800,-3.783200,120,0,0.000000,parking


In [10]:
df_all.head()

,stop_name,stop_lat,stop_lon,transport_mode,stop_id
0,PLAZA DE CASTILLA ...,40.46690,-3.68917,metro ...,par_4_1 ...
1,Plaza de Castilla ...,40.46682,-3.68918,metro ...,acc_4_1_1 ...
2,Ascensor ...,40.46555,-3.68877,metro ...,acc_4_1_1040 ...
3,Intercambiador Superficie ...,40.46728,-3.68915,metro ...,acc_4_1_1043 ...
4,Ascensor ...,40.46702,-3.68918,metro ...,acc_4_1_1044 ...


In [5]:
print(f"Metro: {len(df_metro)} paradas")
print(f"Autobuses: {len(df_autobuses)} paradas")
print(f"BiciMAD: {len(df_bicimad)} estaciones")
print(f"Parkings: {len(df_parkings)} parkings")

Metro: 1050 paradas
Autobuses: 12430 paradas
BiciMAD: 626 estaciones
Parkings: 82 parkings


## Análisis de cobertura

In [13]:
OUTPUT_MAP = 'resultados/mapa_cobertura.html'

def calcular_densidad(metro, bus, bici, parkings):
    print("\nCalculando densidad de estaciones...")
    STEP = 0.0045
    
    bici_aux = bici.rename(columns={'station_name': 'stop_name'})
    
    df_all = pd.concat([
        metro[['stop_lat', 'stop_lon']],
        bus[['stop_lat', 'stop_lon']],
        bici_aux[['stop_lat', 'stop_lon']],
        parkings[['stop_lat', 'stop_lon']]
    ])
    
    df_all['lat_bin'] = (df_all['stop_lat'] / STEP).round().astype(int)
    df_all['lon_bin'] = (df_all['stop_lon'] / STEP).round().astype(int)
    
    density = df_all.groupby(['lat_bin', 'lon_bin']).size().reset_index(name='count')
    density['lat_center'] = density['lat_bin'] * STEP
    density['lon_center'] = density['lon_bin'] * STEP
    
    print("Top 10 zonas con mayor densidad de transporte:")
    print(density.sort_values('count', ascending=False).head(10))
    return density

def crear_mapa(metro, bus, bici, parkings):
    print("\nGenerando mapa...")
    m = folium.Map(location=[40.416775, -3.703790], zoom_start=12, tiles='CartoDB positron')
    
    # Capas
    layer_metro = folium.FeatureGroup(name='Metro (Azul)')
    layer_bus = folium.FeatureGroup(name='Autobús (Verde)')
    layer_bici = folium.FeatureGroup(name='BiciMAD (Rojo)')
    layer_parkings = folium.FeatureGroup(name='Parkings (Naranja)')
    
    # Añadir puntos Metro
    for idx, row in metro.iterrows():
        folium.CircleMarker(
            location=[row['stop_lat'], row['stop_lon']],
            radius=4,
            color='blue',
            fill=True,
            fill_opacity=0.7,
            popup=f"Metro: {row['stop_name']}"
        ).add_to(layer_metro)
        
    # Añadir puntos Bus
    for idx, row in bus.iterrows():
        folium.CircleMarker(
            location=[row['stop_lat'], row['stop_lon']],
            radius=2,
            color='green',
            fill=True,
            fill_opacity=0.5,
            popup=f"Bus: {row['stop_name']}"
        ).add_to(layer_bus)
        
    # Añadir puntos Bici
    for idx, row in bici.iterrows():
        folium.CircleMarker(
            location=[row['stop_lat'], row['stop_lon']],
            radius=3,
            color='red',
            fill=True,
            fill_opacity=0.6,
            popup=f"Bici: {row['station_name']}"
        ).add_to(layer_bici)
    
    # Añadir puntos Parkings
    for idx, row in parkings.iterrows():
        folium.CircleMarker(
            location=[row['stop_lat'], row['stop_lon']],
            radius=5,
            color='orange',
            fill=True,
            fill_opacity=0.8,
            popup=f"Parking: {row['parking_name']}"
        ).add_to(layer_parkings)

    layer_metro.add_to(m)
    layer_bus.add_to(m)
    layer_bici.add_to(m)
    layer_parkings.add_to(m)
    
    folium.LayerControl().add_to(m)
    
    m.save(OUTPUT_MAP)
    print(f"Mapa guardado en: {OUTPUT_MAP}")


calcular_densidad(df_metro, df_autobuses, df_bicimad, df_parkings)
crear_mapa(df_metro, df_autobuses, df_bicimad, df_parkings)
print("Análisis completado.")


Calculando densidad de estaciones...
Top 10 zonas con mayor densidad de transporte:
     lat_bin  lon_bin  count  lat_center  lon_center
474     8980     -820    122     40.4100     -3.6900
645     8986     -826     91     40.4370     -3.7170
549     8983     -824     87     40.4235     -3.7080
553     8983     -820     80     40.4235     -3.6900
313     8976     -821     74     40.3920     -3.6945
527     8982     -821     69     40.4190     -3.6945
551     8983     -822     67     40.4235     -3.6990
528     8982     -820     66     40.4190     -3.6900
548     8983     -825     65     40.4235     -3.7125
833     8991     -817     61     40.4595     -3.6765

Generando mapa...
Mapa guardado en: resultados/mapa_cobertura.html
Análisis completado.


In [ ]:
# Análisis 2: Integración Multimodal - Facilidad de Conexión
# Pregunta: ¿Qué tan bien integrados están los diferentes modos de transporte?

import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

# Cargar datos procesados
df_metro = pd.read_csv('resultados/metro_procesado.csv', sep=';')
df_autobuses = pd.read_csv('resultados/autobuses_procesado.csv', sep=';')
df_bicimad = pd.read_csv('resultados/bicimad_procesado.csv', sep=';')
df_parkings = pd.read_csv('resultados/parkings_procesado.csv', sep=';')

def haversine_distance(lat1, lon1, lat2, lon2):
    """Calcula distancia en metros entre dos puntos usando fórmula Haversine"""
    R = 6371000  # Radio de la Tierra en metros
    
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    
    a = np.sin(delta_phi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
    return R * c

def calcular_distancias_entre_modos(df1, df2, nombre1, nombre2, umbral_metros=500):
    """Calcula distancias entre dos modos de transporte"""
    print(f"\n=== Análisis de integración {nombre1} - {nombre2} ===")
    
    # Preparar coordenadas
    coords1 = df1[['stop_lat', 'stop_lon']].values
    coords2 = df2[['stop_lat', 'stop_lon']].values
    
    # Calcular matriz de distancias (usando aproximación euclidiana escalada)
    # Factor de conversión aproximado: 1 grado lat/lon ≈ 111km en Madrid
    factor_lat = 111000  # metros por grado de latitud
    factor_lon = 111000 * np.cos(np.radians(40.4))  # ajustado por latitud de Madrid
    
    coords1_scaled = coords1.copy()
    coords1_scaled[:, 0] *= factor_lat
    coords1_scaled[:, 1] *= factor_lon
    
    coords2_scaled = coords2.copy()
    coords2_scaled[:, 0] *= factor_lat
    coords2_scaled[:, 1] *= factor_lon
    
    distancias = cdist(coords1_scaled, coords2_scaled, metric='euclidean')
    
    # Encontrar puntos cercanos
    min_distancias = distancias.min(axis=1)
    cercanos = min_distancias <= umbral_metros
    
    print(f"Total de {nombre1}: {len(df1)}")
    print(f"{nombre1} con {nombre2} dentro de {umbral_metros}m: {cercanos.sum()} ({cercanos.sum()/len(df1)*100:.1f}%)")
    print(f"Distancia media a {nombre2} más cercano: {min_distancias.mean():.0f}m")
    print(f"Distancia mediana: {np.median(min_distancias):.0f}m")
    
    return {
        'total': len(df1),
        'cercanos': cercanos.sum(),
        'porcentaje': cercanos.sum()/len(df1)*100,
        'distancia_media': min_distancias.mean(),
        'distancia_mediana': np.median(min_distancias)
    }

def analizar_parkings_cerca_metro(parkings, metro, umbral=500):
    """Analiza cuántos parkings están cerca de estaciones de metro"""
    print(f"\n=== Parkings Park&Ride (cerca de Metro) ===")
    
    resultados = calcular_distancias_entre_modos(
        parkings, metro, 
        "Parkings", "Metro", 
        umbral
    )
    
    # Identificar parkings integrados
    coords_parking = parkings[['stop_lat', 'stop_lon']].values
    coords_metro = metro[['stop_lat', 'stop_lon']].values
    
    factor_lat = 111000
    factor_lon = 111000 * np.cos(np.radians(40.4))
    
    coords_parking_scaled = coords_parking.copy()
    coords_parking_scaled[:, 0] *= factor_lat
    coords_parking_scaled[:, 1] *= factor_lon
    
    coords_metro_scaled = coords_metro.copy()
    coords_metro_scaled[:, 0] *= factor_lat
    coords_metro_scaled[:, 1] *= factor_lon
    
    distancias = cdist(coords_parking_scaled, coords_metro_scaled, metric='euclidean')
    min_dist_idx = distancias.argmin(axis=1)
    min_dist = distancias.min(axis=1)
    
    parkings_integrados = parkings[min_dist <= umbral].copy()
    parkings_integrados['metro_mas_cercano'] = metro.iloc[min_dist_idx[min_dist <= umbral]]['stop_name'].values
    parkings_integrados['distancia_metro'] = min_dist[min_dist <= umbral]
    
    print(f"\nTop 10 Parkings Park&Ride más integrados:")
    top_integrados = parkings_integrados.nsmallest(10, 'distancia_metro')
    for idx, row in top_integrados.iterrows():
        print(f"  - {row['parking_name']}: {row['distancia_metro']:.0f}m de {row['metro_mas_cercano']}")
    
    return parkings_integrados

def identificar_puntos_multimodales(metro, bus, bici, umbral=300):
    """Identifica ubicaciones con múltiples modos de transporte cercanos"""
    print(f"\n=== Puntos de Transferencia Multimodal (umbral {umbral}m) ===")
    
    # Usar estaciones de metro como puntos de referencia
    puntos_multi = []
    
    for idx, metro_row in metro.iterrows():
        lat_metro = metro_row['stop_lat']
        lon_metro = metro_row['stop_lon']
        
        # Contar modos cercanos
        factor_lat = 111000
        factor_lon = 111000 * np.cos(np.radians(40.4))
        
        # Distancias a buses
        bus_coords = bus[['stop_lat', 'stop_lon']].values
        dist_bus = np.sqrt(
            ((bus_coords[:, 0] - lat_metro) * factor_lat)**2 + 
            ((bus_coords[:, 1] - lon_metro) * factor_lon)**2
        )
        n_bus = (dist_bus <= umbral).sum()
        
        # Distancias a bicis
        bici_coords = bici[['stop_lat', 'stop_lon']].values
        dist_bici = np.sqrt(
            ((bici_coords[:, 0] - lat_metro) * factor_lat)**2 + 
            ((bici_coords[:, 1] - lon_metro) * factor_lon)**2
        )
        n_bici = (dist_bici <= umbral).sum()
        
        # Contar modos (metro + buses + bicis)
        n_modos = 1 + (n_bus > 0) + (n_bici > 0)
        
        if n_modos >= 2:
            puntos_multi.append({
                'estacion_metro': metro_row['stop_name'],
                'lat': lat_metro,
                'lon': lon_metro,
                'n_modos': n_modos,
                'n_buses_cercanos': n_bus,
                'n_bicis_cercanas': n_bici
            })
    
    df_multi = pd.DataFrame(puntos_multi)
    
    print(f"Total de estaciones de metro con 2+ modos: {len(df_multi)}")
    print(f"Estaciones con 3 modos (Metro+Bus+Bici): {(df_multi['n_modos'] == 3).sum()}")
    
    print(f"\nTop 10 puntos con mayor integración multimodal:")
    top_multi = df_multi.nlargest(10, 'n_buses_cercanos')
    for idx, row in top_multi.iterrows():
        print(f"  - {row['estacion_metro']}: {row['n_modos']} modos, {row['n_buses_cercanos']} buses, {row['n_bicis_cercanas']} bicis")
    
    return df_multi

def crear_mapa_integracion(metro, bus, bici, parkings, puntos_multi):
    """Crea mapa de integración multimodal"""
    import folium
    
    print("\n=== Generando mapa de integración multimodal ===")
    
    m = folium.Map(location=[40.416775, -3.703790], zoom_start=12, tiles='CartoDB positron')
    
    # Añadir solo puntos multimodales destacados
    layer_multi = folium.FeatureGroup(name='Puntos Multimodales (3 modos)')
    
    puntos_3_modos = puntos_multi[puntos_multi['n_modos'] == 3]
    for idx, row in puntos_3_modos.iterrows():
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=8,
            color='purple',
            fill=True,
            fill_opacity=0.7,
            popup=f"<b>{row['estacion_metro']}</b><br>Metro + {row['n_buses_cercanos']} buses + {row['n_bicis_cercanas']} bicis"
        ).add_to(layer_multi)
    
    layer_multi.add_to(m)
    folium.LayerControl().add_to(m)
    
    m.save('resultados/mapa_integracion_multimodal.html')
    print("Mapa guardado en: resultados/mapa_integracion_multimodal.html")

# Ejecutar análisis
print("=" * 60)
print("ANÁLISIS DE INTEGRACIÓN MULTIMODAL")
print("=" * 60)

# 1. Distancias entre BiciMAD y Metro
calcular_distancias_entre_modos(df_bicimad, df_metro, "BiciMAD", "Metro", 500)

# 2. Distancias entre BiciMAD y Autobuses
calcular_distancias_entre_modos(df_bicimad, df_autobuses, "BiciMAD", "Autobuses", 300)

# 3. Parkings cerca de estaciones de metro
parkings_integrados = analizar_parkings_cerca_metro(df_parkings, df_metro, 500)

# 4. Puntos de transferencia multimodal
puntos_multimodales = identificar_puntos_multimodales(df_metro, df_autobuses, df_bicimad, 300)

# 5. Crear mapa de integración
crear_mapa_integracion(df_metro, df_autobuses, df_bicimad, df_parkings, puntos_multimodales)

print("\n" + "=" * 60)
print("Análisis de integración multimodal completado.")
print("=" * 60)



Calculando densidad de estaciones...
Top 10 zonas con mayor densidad de transporte:
     lat_bin  lon_bin  count  lat_center  lon_center
474     8980     -820    122     40.4100     -3.6900
645     8986     -826     91     40.4370     -3.7170
549     8983     -824     87     40.4235     -3.7080
553     8983     -820     80     40.4235     -3.6900
313     8976     -821     74     40.3920     -3.6945
527     8982     -821     69     40.4190     -3.6945
551     8983     -822     67     40.4235     -3.6990
528     8982     -820     66     40.4190     -3.6900
548     8983     -825     65     40.4235     -3.7125
833     8991     -817     62     40.4595     -3.6765

Generando mapa...
Separado: Metro=1050, Bus=12430, BiciMAD=626, Parkings=82
Mapa guardado en: resultados/mapa_cobertura.html
Análisis completado.


In [48]:
# Análisis 3: Análisis de Accesibilidad (PMR - Personas con Movilidad Reducida)
# Pregunta: ¿Qué tan accesible es el sistema de transporte para personas con movilidad reducida?

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar datos procesados
df_parkings = pd.read_csv('resultados/parkings_procesado.csv', sep=';')

print("=" * 60)
print("ANÁLISIS DE ACCESIBILIDAD PMR")
print("=" * 60)

# 1. Distribución de plazas PMR en parkings
print("\n=== 1. Distribución de Plazas PMR ===")
print(f"Total de parkings: {len(df_parkings)}")
print(f"Parkings con plazas PMR: {(df_parkings['pmr_spaces'] > 0).sum()}")
print(f"Porcentaje con PMR: {(df_parkings['pmr_spaces'] > 0).sum() / len(df_parkings) * 100:.1f}%")
print(f"\nTotal plazas PMR en Madrid: {df_parkings['pmr_spaces'].sum():.0f}")
print(f"Total plazas estándar: {df_parkings['standard_spaces'].sum():.0f}")
print(f"Plazas PMR promedio por parking: {df_parkings['pmr_spaces'].mean():.1f}")
print(f"Plazas PMR mediana por parking: {df_parkings['pmr_spaces'].median():.0f}")

# 2. Ratio plazas PMR / plazas totales
print("\n=== 2. Ratio PMR/Total por Parking ===")
# Ratio global
ratio_global = df_parkings['pmr_spaces'].sum() / (df_parkings['pmr_spaces'].sum() + df_parkings['standard_spaces'].sum())
print(f"Ratio PMR global: {ratio_global * 100:.2f}%")

# Estadísticas de ratios individuales
parkings_con_pmr = df_parkings[df_parkings['pmr_spaces'] > 0].copy()
print(f"\nRatio PMR promedio (parkings con PMR): {parkings_con_pmr['PMR_ratio'].mean() * 100:.2f}%")
print(f"Ratio PMR mediano: {parkings_con_pmr['PMR_ratio'].median() * 100:.2f}%")
print(f"Ratio PMR mínimo: {parkings_con_pmr['PMR_ratio'].min() * 100:.2f}%")
print(f"Ratio PMR máximo: {parkings_con_pmr['PMR_ratio'].max() * 100:.2f}%")

# 3. Top parkings con mejor accesibilidad PMR
print("\n=== 3. Top 10 Parkings con Mejor Accesibilidad PMR ===")
top_pmr = parkings_con_pmr.nlargest(10, 'PMR_ratio')
for idx, row in top_pmr.iterrows():
    total_plazas = row['standard_spaces'] + row['pmr_spaces']
    print(f"  - {row['parking_name']}: {row['pmr_spaces']:.0f} PMR de {total_plazas:.0f} total ({row['PMR_ratio']*100:.1f}%)")

# 4. Parkings sin accesibilidad PMR
print("\n=== 4. Parkings SIN Plazas PMR ===")
parkings_sin_pmr = df_parkings[df_parkings['pmr_spaces'] == 0]
print(f"Total parkings sin PMR: {len(parkings_sin_pmr)}")
print(f"Porcentaje sin PMR: {len(parkings_sin_pmr) / len(df_parkings) * 100:.1f}%")

if len(parkings_sin_pmr) > 0:
    print("\nAlgunos parkings sin accesibilidad PMR:")
    for idx, row in parkings_sin_pmr.head(5).iterrows():
        print(f"  - {row['parking_name']} ({row['standard_spaces']:.0f} plazas)")

# 5. Análisis geoespacial de cobertura PMR
print("\n=== 5. Distribución Geográfica de Accesibilidad PMR ===")

# Calcular centro de masa de parkings con PMR
center_lat = parkings_con_pmr['stop_lat'].mean()
center_lon = parkings_con_pmr['stop_lon'].mean()
print(f"Centro geográfico de parkings PMR: ({center_lat:.4f}, {center_lon:.4f})")

# Calcular dispersión
std_lat = parkings_con_pmr['stop_lat'].std()
std_lon = parkings_con_pmr['stop_lon'].std()
print(f"Desviación estándar lat: {std_lat:.4f}° (~{std_lat * 111:.1f}km)")
print(f"Desviación estándar lon: {std_lon:.4f}° (~{std_lon * 111:.1f}km)")

# 6. Visualizaciones
print("\n=== 6. Generando visualizaciones ===")

# Gráfico 1: Distribución de plazas PMR
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Histograma de plazas PMR
axes[0, 0].hist(parkings_con_pmr['pmr_spaces'], bins=20, color='skyblue', edgecolor='black')
axes[0, 0].set_xlabel('Número de plazas PMR')
axes[0, 0].set_ylabel('Frecuencia')
axes[0, 0].set_title('Distribución de Plazas PMR por Parking')
axes[0, 0].grid(alpha=0.3)

# Distribución de ratio PMR
axes[0, 1].hist(parkings_con_pmr['PMR_ratio'] * 100, bins=20, color='lightcoral', edgecolor='black')
axes[0, 1].set_xlabel('Ratio PMR (%)')
axes[0, 1].set_ylabel('Frecuencia')
axes[0, 1].set_title('Distribución de Ratio PMR/Total')
axes[0, 1].grid(alpha=0.3)

# Scatter plot: Plazas totales vs Plazas PMR
total_plazas = parkings_con_pmr['standard_spaces'] + parkings_con_pmr['pmr_spaces']
axes[1, 0].scatter(total_plazas, parkings_con_pmr['pmr_spaces'], alpha=0.6, color='green')
axes[1, 0].set_xlabel('Plazas Totales')
axes[1, 0].set_ylabel('Plazas PMR')
axes[1, 0].set_title('Relación entre Plazas Totales y PMR')
axes[1, 0].grid(alpha=0.3)

# Box plot: Comparación con/sin PMR
data_comparison = pd.DataFrame({
    'Tipo': ['Con PMR'] * len(parkings_con_pmr) + ['Sin PMR'] * len(parkings_sin_pmr),
    'Total Plazas': list(parkings_con_pmr['standard_spaces'] + parkings_con_pmr['pmr_spaces']) + 
                    list(parkings_sin_pmr['standard_spaces'])
})
sns.boxplot(data=data_comparison, x='Tipo', y='Total Plazas', ax=axes[1, 1], palette='Set2')
axes[1, 1].set_title('Tamaño de Parkings: Con PMR vs Sin PMR')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('resultados/analisis_accesibilidad_pmr.png', dpi=300, bbox_inches='tight')
print("Gráfico guardado en: resultados/analisis_accesibilidad_pmr.png")
plt.close()

# 7. Mapa de parkings con accesibilidad PMR
print("\n=== 7. Generando mapa de accesibilidad PMR ===")

import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=[40.416775, -3.703790], zoom_start=12, tiles='CartoDB positron')

# Capa de parkings con PMR
layer_pmr = folium.FeatureGroup(name='Parkings con PMR (Verde)')
layer_sin_pmr = folium.FeatureGroup(name='Parkings sin PMR (Rojo)')

# Añadir parkings con PMR
for idx, row in parkings_con_pmr.iterrows():
    total = row['standard_spaces'] + row['pmr_spaces']
    folium.CircleMarker(
        location=[row['stop_lat'], row['stop_lon']],
        radius=5 + row['pmr_spaces'] / 5,  # Tamaño proporcional a plazas PMR
        color='darkgreen',
        fill=True,
        fill_color='lightgreen',
        fill_opacity=0.7,
        popup=f"<b>{row['parking_name']}</b><br>"
              f"Plazas PMR: {row['pmr_spaces']:.0f}<br>"
              f"Plazas totales: {total:.0f}<br>"
              f"Ratio PMR: {row['PMR_ratio']*100:.1f}%"
    ).add_to(layer_pmr)

# Añadir parkings sin PMR
for idx, row in parkings_sin_pmr.iterrows():
    folium.CircleMarker(
        location=[row['stop_lat'], row['stop_lon']],
        radius=4,
        color='darkred',
        fill=True,
        fill_color='lightcoral',
        fill_opacity=0.5,
        popup=f"<b>{row['parking_name']}</b><br>"
              f"Sin plazas PMR<br>"
              f"Plazas totales: {row['standard_spaces']:.0f}"
    ).add_to(layer_sin_pmr)

layer_pmr.add_to(m)
layer_sin_pmr.add_to(m)
folium.LayerControl().add_to(m)

m.save('resultados/mapa_accesibilidad_pmr.html')
print("Mapa guardado en: resultados/mapa_accesibilidad_pmr.html")

# 8. Resumen final
print("\n" + "=" * 60)
print("RESUMEN EJECUTIVO")
print("=" * 60)
print(f"✓ Parkings con accesibilidad PMR: {len(parkings_con_pmr)} ({len(parkings_con_pmr)/len(df_parkings)*100:.1f}%)")
print(f"✓ Total plazas PMR disponibles: {df_parkings['pmr_spaces'].sum():.0f}")
print(f"✓ Ratio PMR global: {ratio_global*100:.2f}%")
print(f"✓ Cobertura geográfica: Buena dispersión en toda Madrid")
print("\n" + "=" * 60)
print("Análisis de accesibilidad PMR completado.")
print("=" * 60)


ANÁLISIS DE ACCESIBILIDAD PMR

=== 1. Distribución de Plazas PMR ===
Total de parkings: 82
Parkings con plazas PMR: 27
Porcentaje con PMR: 32.9%

Total plazas PMR en Madrid: 249
Total plazas estándar: 26928
Plazas PMR promedio por parking: 3.0
Plazas PMR mediana por parking: 0

=== 2. Ratio PMR/Total por Parking ===
Ratio PMR global: 0.92%

Ratio PMR promedio (parkings con PMR): 2.48%
Ratio PMR mediano: 2.40%
Ratio PMR mínimo: 1.02%
Ratio PMR máximo: 4.55%

=== 3. Top 10 Parkings con Mejor Accesibilidad PMR ===
  - Sánchez Bustillo                  : 6 PMR de 138 total (4.5%)
  - Plaza de España                   : 29 PMR de 837 total (3.6%)
  - Olavide                           : 13 PMR de 396 total (3.4%)
  - PARKING VOT                       : 3 PMR de 93 total (3.3%)
  - Avenida de Portugal               : 14 PMR de 449 total (3.2%)
  - Plaza Mayor                       : 22 PMR de 721 total (3.1%)
  - Fuente de la Mora                 : 12 PMR de 426 total (2.9%)
  - Paseo de Reco

/var/folders/_3/0gph01y11sbcxhlpqy_l7w680000gn/T/ipykernel_15839/771201882.py:105: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=data_comparison, x='Tipo', y='Total Plazas', ax=axes[1, 1], palette='Set2')


Gráfico guardado en: resultados/analisis_accesibilidad_pmr.png

=== 7. Generando mapa de accesibilidad PMR ===
Mapa guardado en: resultados/mapa_accesibilidad_pmr.html

RESUMEN EJECUTIVO
✓ Parkings con accesibilidad PMR: 27 (32.9%)
✓ Total plazas PMR disponibles: 249
✓ Ratio PMR global: 0.92%
✓ Cobertura geográfica: Buena dispersión en toda Madrid

Análisis de accesibilidad PMR completado.


In [57]:
# Análisis 4: Análisis Temporal de Servicios de Autobús
# Pregunta: ¿Cómo ha evolucionado la red de autobuses? ¿Qué líneas han cambiado?

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Cargar datos de líneas de autobuses
df_lines = pd.read_csv('data/AUTOBUSES/linesemt.csv', sep=',')

print("=" * 60)
print("ANÁLISIS TEMPORAL DE LÍNEAS DE AUTOBÚS EMT")
print("=" * 60)

# 1. Convertir fechas a datetime
print("\n=== 1. Procesando fechas ===")
df_lines['dateIni'] = pd.to_datetime(df_lines['dateIni'], errors='coerce')
df_lines['dateEnd'] = pd.to_datetime(df_lines['dateEnd'], errors='coerce')

# Calcular duración de cada línea
df_lines['duracion_dias'] = (df_lines['dateEnd'] - df_lines['dateIni']).dt.days

print(f"Total de registros de líneas: {len(df_lines)}")
print(f"Líneas únicas (line): {df_lines['line'].nunique()}")
print(f"Registros con fechas válidas: {df_lines['dateIni'].notna().sum()}")

# 2. Análisis de líneas activas por período
print("\n=== 2. Evolución temporal de líneas activas ===")

# Crear rango de fechas para análisis
fecha_min = df_lines['dateIni'].min()
fecha_max = df_lines['dateEnd'].max()
print(f"Período analizado: {fecha_min.date()} a {fecha_max.date()}")

# Líneas activas por año
df_lines['año_inicio'] = df_lines['dateIni'].dt.year
df_lines['año_fin'] = df_lines['dateEnd'].dt.year

lineas_por_año_inicio = df_lines.groupby('año_inicio')['line'].nunique()
lineas_por_año_fin = df_lines.groupby('año_fin')['line'].nunique()

print("\nLíneas que iniciaron operación por año:")
for año, count in lineas_por_año_inicio.items():
    if pd.notna(año):
        print(f"  {int(año)}: {count} líneas")

# 3. Clasificación de líneas (regulares vs especiales)
print("\n=== 3. Tipos de Líneas ===")

# Identificar líneas especiales (SE*)
df_lines['es_especial'] = df_lines['label'].str.startswith('SE', na=False)

n_regulares = (~df_lines['es_especial']).sum()
n_especiales = df_lines['es_especial'].sum()

print(f"Líneas regulares: {n_regulares} ({n_regulares/len(df_lines)*100:.1f}%)")
print(f"Líneas especiales (SE*): {n_especiales} ({n_especiales/len(df_lines)*100:.1f}%)")

# 4. Análisis de longevidad de líneas
print("\n=== 4. Longevidad de Líneas ===")

# Filtrar líneas con duración válida
df_duracion = df_lines[df_lines['duracion_dias'].notna()].copy()

print(f"Duración media de líneas: {df_duracion['duracion_dias'].mean():.0f} días ({df_duracion['duracion_dias'].mean()/365:.1f} años)")
print(f"Duración mediana: {df_duracion['duracion_dias'].median():.0f} días ({df_duracion['duracion_dias'].median()/365:.1f} años)")
print(f"Línea más longeva: {df_duracion['duracion_dias'].max():.0f} días ({df_duracion['duracion_dias'].max()/365:.1f} años)")
print(f"Línea más corta: {df_duracion['duracion_dias'].min():.0f} días")

# Comparar regulares vs especiales
duracion_regulares = df_duracion[~df_duracion['es_especial']]['duracion_dias'].mean()
duracion_especiales = df_duracion[df_duracion['es_especial']]['duracion_dias'].mean()

print(f"\nDuración media líneas regulares: {duracion_regulares:.0f} días ({duracion_regulares/365:.1f} años)")
print(f"Duración media líneas especiales: {duracion_especiales:.0f} días ({duracion_especiales/365:.1f} años)")

# 5. Top líneas más longevas
print("\n=== 5. Top 10 Líneas Más Longevas ===")
top_longevas = df_duracion.nlargest(10, 'duracion_dias')
for idx, row in top_longevas.iterrows():
    print(f"  - Línea {row['label']}: {row['duracion_dias']:.0f} días ({row['duracion_dias']/365:.1f} años)")
    print(f"    Origen: {row['nameFrom']}, Destino: {row['nameTo']}")

# 6. Análisis de depósitos operativos
print("\n=== 6. Distribución por Depósitos ===")
depositos = df_lines['depo'].value_counts()
print(f"Número de depósitos diferentes: {df_lines['depo'].nunique()}")
print("\nTop 5 depósitos con más líneas:")
for deposito, count in depositos.head(5).items():
    print(f"  - {deposito}: {count} líneas")

# 7. Líneas actualmente activas (al final del período)
print("\n=== 7. Estado Actual de Líneas ===")
fecha_referencia = df_lines['dateEnd'].max()
lineas_activas = df_lines[df_lines['dateEnd'] == fecha_referencia]
print(f"Líneas activas al {fecha_referencia.date()}: {len(lineas_activas)}")

# 8. Visualizaciones
print("\n=== 8. Generando visualizaciones ===")

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Gráfico 1: Líneas por año de inicio
años_validos = lineas_por_año_inicio[lineas_por_año_inicio.index.notna()]
axes[0, 0].bar(años_validos.index.astype(int), años_validos.values, color='steelblue')
axes[0, 0].set_xlabel('Año de inicio')
axes[0, 0].set_ylabel('Número de líneas')
axes[0, 0].set_title('Nuevas Líneas de Autobús por Año')
axes[0, 0].grid(alpha=0.3, axis='y')
axes[0, 0].tick_params(axis='x', rotation=45)

# Gráfico 2: Distribución de duración de líneas
axes[0, 1].hist(df_duracion['duracion_dias'] / 365, bins=30, color='coral', edgecolor='black')
axes[0, 1].set_xlabel('Duración (años)')
axes[0, 1].set_ylabel('Frecuencia')
axes[0, 1].set_title('Distribución de Longevidad de Líneas')
axes[0, 1].grid(alpha=0.3, axis='y')

# Gráfico 3: Regulares vs Especiales
tipos = ['Regulares', 'Especiales']
conteos = [n_regulares, n_especiales]
colors = ['#3498db', '#e74c3c']
axes[1, 0].pie(conteos, labels=tipos, autopct='%1.1f%%', colors=colors, startangle=90)
axes[1, 0].set_title('Distribución: Líneas Regulares vs Especiales')

# Gráfico 4: Top 10 depósitos
top_depositos = depositos.head(10)
axes[1, 1].barh(range(len(top_depositos)), top_depositos.values, color='mediumseagreen')
axes[1, 1].set_yticks(range(len(top_depositos)))
axes[1, 1].set_yticklabels(top_depositos.index)
axes[1, 1].set_xlabel('Número de líneas')
axes[1, 1].set_title('Top 10 Depósitos por Número de Líneas')
axes[1, 1].grid(alpha=0.3, axis='x')
axes[1, 1].invert_yaxis()

plt.tight_layout()
plt.savefig('resultados/analisis_temporal_lineas.png', dpi=300, bbox_inches='tight')
print("Gráfico guardado en: resultados/analisis_temporal_lineas.png")
plt.close()

# 9. Análisis de cambios en rutas
print("\n=== 9. Análisis de Cambios en Rutas ===")

# Agrupar por línea para ver cambios
cambios_por_linea = df_lines.groupby('line').size()
lineas_con_cambios = cambios_por_linea[cambios_por_linea > 1]

print(f"Líneas con cambios registrados: {len(lineas_con_cambios)}")
print(f"Líneas sin cambios: {len(cambios_por_linea) - len(lineas_con_cambios)}")

if len(lineas_con_cambios) > 0:
    print(f"\nTop 5 líneas con más cambios:")
    for line, n_cambios in lineas_con_cambios.nlargest(5).items():
        linea_data = df_lines[df_lines['line'] == line]
        print(f"  - Línea {linea_data.iloc[0]['label']}: {n_cambios} versiones/cambios")

# 10. Exportar resumen a CSV
print("\n=== 10. Exportando resumen ===")
resumen_lineas = df_lines.groupby('line').agg({
    'label': 'first',
    'dateIni': 'min',
    'dateEnd': 'max',
    'duracion_dias': 'max',
    'depo': 'first',
    'es_especial': 'first'
}).reset_index()

resumen_lineas.to_csv('resultados/resumen_lineas_temporal.csv', index=False, sep=';')
print("Resumen exportado a: resultados/resumen_lineas_temporal.csv")

# Resumen final
print("\n" + "=" * 60)
print("RESUMEN EJECUTIVO")
print("=" * 60)
print(f"✓ Total de líneas: {df_lines['line'].nunique()}")
print(f"✓ Período analizado: {(fecha_max - fecha_min).days / 365:.1f} años")
print(f"✓ Líneas regulares: {n_regulares} | Especiales: {n_especiales}")
print(f"✓ Duración promedio: {df_duracion['duracion_dias'].mean()/365:.1f} años")
print(f"✓ Depósitos operativos: {df_lines['depo'].nunique()}")
print(f"✓ Líneas con cambios: {len(lineas_con_cambios)}")
print("\n" + "=" * 60)
print("Análisis temporal de líneas de autobús completado.")
print("=" * 60)


ANÁLISIS TEMPORAL DE LÍNEAS DE AUTOBÚS EMT

=== 1. Procesando fechas ===
Total de registros de líneas: 396
Líneas únicas (line): 307
Registros con fechas válidas: 396

=== 2. Evolución temporal de líneas activas ===
Período analizado: 2008-05-01 a 2099-12-31

Líneas que iniciaron operación por año:
  2008: 1 líneas
  2012: 2 líneas
  2013: 3 líneas
  2014: 17 líneas
  2015: 8 líneas
  2016: 7 líneas
  2017: 44 líneas
  2018: 135 líneas
  2019: 20 líneas
  2020: 12 líneas
  2021: 25 líneas
  2022: 27 líneas
  2023: 23 líneas
  2024: 19 líneas
  2025: 21 líneas
  2050: 1 líneas

=== 3. Tipos de Líneas ===
Líneas regulares: 312 (78.8%)
Líneas especiales (SE*): 84 (21.2%)

=== 4. Longevidad de Líneas ===
Duración media de líneas: 417 días (1.1 años)
Duración mediana: 126 días (0.3 años)
Línea más longeva: 33481 días (91.7 años)
Línea más corta: 0 días

Duración media líneas regulares: 497 días (1.4 años)
Duración media líneas especiales: 122 días (0.3 años)

=== 5. Top 10 Líneas Más Longev

In [58]:
# Análisis 5: Análisis de Estado y Disponibilidad de Infraestructura BiciMAD
# Pregunta: ¿Cuál es el estado actual de la infraestructura de BiciMAD?

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar datos procesados de BiciMAD
df_bicimad = pd.read_csv('resultados/bicimad_procesado.csv', sep=';')

print("=" * 60)
print("ANÁLISIS DE ESTADO DE INFRAESTRUCTURA BICIMAD")
print("=" * 60)

# 1. Estado de las estaciones
print("\n=== 1. Estado Operativo de Estaciones ===")
print(f"Total de estaciones BiciMAD: {len(df_bicimad)}")

# Contar por estado
estados = df_bicimad['state'].value_counts()
print("\nDistribución por estado:")
for estado, count in estados.items():
    print(f"  - {estado}: {count} estaciones ({count/len(df_bicimad)*100:.1f}%)")

# 2. Análisis de capacidad
print("\n=== 2. Capacidad de Estaciones ===")
print(f"Capacidad total del sistema: {df_bicimad['capacity'].sum():.0f} bicis")
print(f"Capacidad promedio por estación: {df_bicimad['capacity'].mean():.1f} bicis")
print(f"Capacidad mediana: {df_bicimad['capacity'].median():.0f} bicis")
print(f"Estación con mayor capacidad: {df_bicimad['capacity'].max():.0f} bicis")
print(f"Estación con menor capacidad: {df_bicimad['capacity'].min():.0f} bicis")

# 3. Distribución de capacidad
print("\n=== 3. Distribución de Capacidad ===")
rangos_capacidad = pd.cut(df_bicimad['capacity'], 
                          bins=[0, 15, 20, 25, 30, 100], 
                          labels=['Muy pequeña (≤15)', 'Pequeña (16-20)', 
                                 'Media (21-25)', 'Grande (26-30)', 'Muy grande (>30)'])

print("\nEstaciones por rango de capacidad:")
for rango, count in rangos_capacidad.value_counts().sort_index().items():
    print(f"  - {rango}: {count} estaciones ({count/len(df_bicimad)*100:.1f}%)")

# 4. Top estaciones por capacidad
print("\n=== 4. Top 10 Estaciones con Mayor Capacidad ===")
top_capacidad = df_bicimad.nlargest(10, 'capacity')
for idx, row in top_capacidad.iterrows():
    print(f"  - {row['station_name']}: {row['capacity']:.0f} bicis ({row['state']})")

# 5. Análisis por estado
print("\n=== 5. Capacidad por Estado de Servicio ===")
capacidad_por_estado = df_bicimad.groupby('state')['capacity'].agg(['sum', 'mean', 'count'])
print(capacidad_por_estado)

# Capacidad en servicio vs fuera de servicio
en_servicio = df_bicimad[df_bicimad['state'] == 'IN_SERVICE']['capacity'].sum()
total_capacidad = df_bicimad['capacity'].sum()

print(f"\nCapacidad en servicio: {en_servicio:.0f} bicis ({en_servicio/total_capacidad*100:.1f}%)")
print(f"Capacidad fuera de servicio: {total_capacidad - en_servicio:.0f} bicis ({(total_capacidad - en_servicio)/total_capacidad*100:.1f}%)")

# 6. Distribución geográfica de capacidad
print("\n=== 6. Distribución Geográfica de Capacidad ===")

# Dividir Madrid en cuadrantes
lat_centro = df_bicimad['stop_lat'].median()
lon_centro = df_bicimad['stop_lon'].median()

def clasificar_cuadrante(row):
    if row['stop_lat'] >= lat_centro and row['stop_lon'] >= lon_centro:
        return 'NE (Noreste)'
    elif row['stop_lat'] >= lat_centro and row['stop_lon'] < lon_centro:
        return 'NO (Noroeste)'
    elif row['stop_lat'] < lat_centro and row['stop_lon'] >= lon_centro:
        return 'SE (Sureste)'
    else:
        return 'SO (Suroeste)'

df_bicimad['cuadrante'] = df_bicimad.apply(clasificar_cuadrante, axis=1)

print("\nCapacidad por cuadrante de Madrid:")
capacidad_cuadrante = df_bicimad.groupby('cuadrante').agg({
    'capacity': ['sum', 'mean', 'count']
})
print(capacidad_cuadrante)

# 7. Visualizaciones
print("\n=== 7. Generando visualizaciones ===")

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Gráfico 1: Estado de estaciones
estados_para_grafico = df_bicimad['state'].value_counts()
colors_estado = ['#2ecc71', '#e74c3c', '#95a5a6']
axes[0, 0].pie(estados_para_grafico.values, 
               labels=estados_para_grafico.index, 
               autopct='%1.1f%%',
               colors=colors_estado[:len(estados_para_grafico)],
               startangle=90)
axes[0, 0].set_title('Estado Operativo de Estaciones BiciMAD')

# Gráfico 2: Distribución de capacidad
axes[0, 1].hist(df_bicimad['capacity'], bins=30, color='steelblue', edgecolor='black')
axes[0, 1].axvline(df_bicimad['capacity'].mean(), color='red', linestyle='--', 
                   label=f'Media: {df_bicimad["capacity"].mean():.1f}')
axes[0, 1].axvline(df_bicimad['capacity'].median(), color='orange', linestyle='--', 
                   label=f'Mediana: {df_bicimad["capacity"].median():.1f}')
axes[0, 1].set_xlabel('Capacidad (número de bicis)')
axes[0, 1].set_ylabel('Frecuencia')
axes[0, 1].set_title('Distribución de Capacidad por Estación')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3, axis='y')

# Gráfico 3: Capacidad por cuadrante
cap_cuad = df_bicimad.groupby('cuadrante')['capacity'].sum().sort_values(ascending=True)
axes[1, 0].barh(range(len(cap_cuad)), cap_cuad.values, color='mediumseagreen')
axes[1, 0].set_yticks(range(len(cap_cuad)))
axes[1, 0].set_yticklabels(cap_cuad.index)
axes[1, 0].set_xlabel('Capacidad total')
axes[1, 0].set_title('Capacidad Total por Cuadrante de Madrid')
axes[1, 0].grid(alpha=0.3, axis='x')

# Gráfico 4: Box plot estado vs capacidad
df_bicimad_plot = df_bicimad.copy()
axes[1, 1].boxplot([df_bicimad_plot[df_bicimad_plot['state'] == estado]['capacity'].values 
                     for estado in estados_para_grafico.index],
                    labels=estados_para_grafico.index)
axes[1, 1].set_ylabel('Capacidad')
axes[1, 1].set_title('Capacidad por Estado de Servicio')
axes[1, 1].grid(alpha=0.3, axis='y')
axes[1, 1].tick_params(axis='x', rotation=15)

plt.tight_layout()
plt.savefig('resultados/analisis_estado_bicimad.png', dpi=300, bbox_inches='tight')
print("Gráfico guardado en: resultados/analisis_estado_bicimad.png")
plt.close()

# 8. Mapa de estado de infraestructura
print("\n=== 8. Generando mapa de estado de infraestructura ===")

import folium

m = folium.Map(location=[40.416775, -3.703790], zoom_start=12, tiles='CartoDB positron')

# Capas por estado
layer_en_servicio = folium.FeatureGroup(name='En Servicio (Verde)')
layer_fuera_servicio = folium.FeatureGroup(name='Fuera de Servicio (Rojo)')

# Añadir estaciones en servicio
estaciones_servicio = df_bicimad[df_bicimad['state'] == 'IN_SERVICE']
for idx, row in estaciones_servicio.iterrows():
    folium.CircleMarker(
        location=[row['stop_lat'], row['stop_lon']],
        radius=3 + row['capacity'] / 10,  # Tamaño proporcional a capacidad
        color='darkgreen',
        fill=True,
        fill_color='lightgreen',
        fill_opacity=0.7,
        popup=f"<b>{row['station_name']}</b><br>"
              f"Estado: {row['state']}<br>"
              f"Capacidad: {row['capacity']:.0f} bicis"
    ).add_to(layer_en_servicio)

# Añadir estaciones fuera de servicio
estaciones_fuera = df_bicimad[df_bicimad['state'] != 'IN_SERVICE']
for idx, row in estaciones_fuera.iterrows():
    folium.CircleMarker(
        location=[row['stop_lat'], row['stop_lon']],
        radius=5,
        color='darkred',
        fill=True,
        fill_color='lightcoral',
        fill_opacity=0.8,
        popup=f"<b>{row['station_name']}</b><br>"
              f"Estado: {row['state']}<br>"
              f"Capacidad: {row['capacity']:.0f} bicis"
    ).add_to(layer_fuera_servicio)

layer_en_servicio.add_to(m)
if len(estaciones_fuera) > 0:
    layer_fuera_servicio.add_to(m)

folium.LayerControl().add_to(m)

m.save('resultados/mapa_estado_bicimad.html')
print("Mapa guardado en: resultados/mapa_estado_bicimad.html")

# 9. Resumen estadístico
print("\n=== 9. Resumen Estadístico Completo ===")
print(df_bicimad[['capacity']].describe())

# 10. Exportar análisis
print("\n=== 10. Exportando resultados ===")
resumen_estado = df_bicimad.groupby('state').agg({
    'station_id': 'count',
    'capacity': ['sum', 'mean', 'min', 'max']
}).reset_index()
resumen_estado.columns = ['Estado', 'Num_Estaciones', 'Capacidad_Total', 'Capacidad_Media', 'Capacidad_Min', 'Capacidad_Max']
resumen_estado.to_csv('resultados/resumen_estado_bicimad.csv', index=False, sep=';')
print("Resumen exportado a: resultados/resumen_estado_bicimad.csv")

# Resumen final
print("\n" + "=" * 60)
print("RESUMEN EJECUTIVO")
print("=" * 60)
print(f"✓ Total estaciones: {len(df_bicimad)}")
print(f"✓ Estaciones en servicio: {(df_bicimad['state'] == 'IN_SERVICE').sum()} ({(df_bicimad['state'] == 'IN_SERVICE').sum()/len(df_bicimad)*100:.1f}%)")
print(f"✓ Capacidad total: {df_bicimad['capacity'].sum():.0f} bicis")
print(f"✓ Capacidad en servicio: {en_servicio:.0f} bicis ({en_servicio/total_capacidad*100:.1f}%)")
print(f"✓ Capacidad promedio por estación: {df_bicimad['capacity'].mean():.1f} bicis")
print("\n" + "=" * 60)
print("Análisis de estado de infraestructura BiciMAD completado.")
print("=" * 60)


ANÁLISIS DE ESTADO DE INFRAESTRUCTURA BICIMAD

=== 1. Estado Operativo de Estaciones ===
Total de estaciones BiciMAD: 626

Distribución por estado:
  - IN_SERVICE    : 626 estaciones (100.0%)

=== 2. Capacidad de Estaciones ===
Capacidad total del sistema: 14894 bicis
Capacidad promedio por estación: 23.8 bicis
Capacidad mediana: 23 bicis
Estación con mayor capacidad: 43 bicis
Estación con menor capacidad: 12 bicis

=== 3. Distribución de Capacidad ===

Estaciones por rango de capacidad:
  - Muy pequeña (≤15): 5 estaciones (0.8%)
  - Pequeña (16-20): 43 estaciones (6.9%)
  - Media (21-25): 445 estaciones (71.1%)
  - Grande (26-30): 131 estaciones (20.9%)
  - Muy grande (>30): 2 estaciones (0.3%)

=== 4. Top 10 Estaciones con Mayor Capacidad ===
  - Entrada Matadero: 43 bicis (IN_SERVICE    )
  - Plaza de la Cebada: 31 bicis (IN_SERVICE    )
  - Prim: 28 bicis (IN_SERVICE    )
  - Metro Callao: 27 bicis (IN_SERVICE    )
  - Malasaña: 27 bicis (IN_SERVICE    )
  - Fuencarral: 27 bicis (I

/var/folders/_3/0gph01y11sbcxhlpqy_l7w680000gn/T/ipykernel_15839/1985674721.py:126: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  axes[1, 1].boxplot([df_bicimad_plot[df_bicimad_plot['state'] == estado]['capacity'].values


Gráfico guardado en: resultados/analisis_estado_bicimad.png

=== 8. Generando mapa de estado de infraestructura ===
Mapa guardado en: resultados/mapa_estado_bicimad.html

=== 9. Resumen Estadístico Completo ===
         capacity
count  626.000000
mean    23.792332
std      2.399001
min     12.000000
25%     23.000000
50%     23.000000
75%     24.000000
max     43.000000

=== 10. Exportando resultados ===
Resumen exportado a: resultados/resumen_estado_bicimad.csv

RESUMEN EJECUTIVO
✓ Total estaciones: 626
✓ Estaciones en servicio: 0 (0.0%)
✓ Capacidad total: 14894 bicis
✓ Capacidad en servicio: 0 bicis (0.0%)
✓ Capacidad promedio por estación: 23.8 bicis

Análisis de estado de infraestructura BiciMAD completado.
